In [1]:
def fasta_iter(fastafile):
    """
    Iterator of the records present in a .fasta file. Each record is yielded as
    a tuple containg the title line (without '>') and the sequence itself.
    """
    on_open_record = False
    with open(fastafile, 'r') as fasta:
        for line in fasta:
            line = line.strip()
            if len(line) == 0:
                continue
            elif line[0] == '#':
                continue
            elif line[0] == '>':
                # close old record
                if on_open_record:
                    yield title, sequence.upper()
                # new record
                title = line[1:].rstrip()
                sequence = ''
                on_open_record = True
            else:
                sequence += line.rstrip()
    if on_open_record:
        yield title, sequence.upper()

# seqs from Uniprot

In [2]:
from uniprobe_data import *

In [4]:
metadata.create_all()
session.commit()

In [9]:
import urllib2

url = 'http://www.uniprot.org/uniprot/'

with open('uniprot_seq.fasta', 'w') as fasta:
    for gene_info in session.query(GeneInfo):
        
        if gene_info.uniprot:
            uniprot_fasta = None
            try:
                uniprot_fasta = urllib2.urlopen(url + gene_info.uniprot + '.fasta').read()
                if uniprot_fasta:
                    __u = uniprot_fasta.split('\n')
                    uniprot_fasta = __u[0] + '\n' + ''.join(__u[1:])
                    fasta.write(uniprot_fasta + '\n')
                    
                    gene_data = gene_info.gene_data
                    if gene_data:
                        gene_data.fasta = uniprot_fasta
                    else:
                        gene_data = GeneData(gene_name=gene_info.gene_name, species=gene_info.species,
                                             name=gene_info.name, synonyms=gene_info.synonyms,
                                             uniprot=gene_info.uniprot, fasta=uniprot_fasta)
                        session.add(gene_data)
                else:
                    print('- %s: empty fasta' % gene_info.uniprot)

            except urllib2.URLError as e:
                print('  %s: %s (%s)' % (gene_info.uniprot, e.code, e.reason))
            
        else:
            print('> %s: no uniprot entry' % gene_info.gene_name)

session.commit()

- Q6LFM5: empty fasta
> Dobox4: no uniprot entry
> Dobox5: no uniprot entry
> Tgif1: no uniprot entry
> Zscan4: no uniprot entry
> IRC900814: no uniprot entry
> Gsm1: no uniprot entry
> Yrr1: no uniprot entry
> Stp4: no uniprot entry
> Ypr196w: no uniprot entry
> Matalpha2: no uniprot entry
> Ydr520c: no uniprot entry
> Yll054c: no uniprot entry
> Usv1: no uniprot entry
> Ypr013c: no uniprot entry
> Ypr015c: no uniprot entry
> Yrm1: no uniprot entry
- Q17588: empty fasta
- P91527: empty fasta
- Q21663: empty fasta
- P90982: empty fasta
- Q22069: empty fasta
- Q4PIU8: empty fasta
- D3Z7Q1: empty fasta
> Gm4881: no uniprot entry
> PmTbr: no uniprot entry
> ACA1_126960: no uniprot entry
> AMAG_02766: no uniprot entry
> AMAG_00796: no uniprot entry
> FoxN1-4_Mbre: no uniprot entry


In [6]:
from sqlalchemy import exists

for gene_data in session.query(GeneData):
    if gene_data.uniprot != gene_data.fasta.split('|')[1]:
        print(gene_data.uniprot, gene_data.fasta.split('|')[1],
              session.query(exists().where(GeneData.uniprot == gene_data.fasta.split('|')[1])).first()[0])

(u'P31312', u'P31257', True)
(u'Q32M12', u'Q9WUN8', False)
(u'Q9D2W8', u'Q3UHX8', False)
(u'P70367', u'Q8VIH1', False)
(u'Q78DU3', u'O08934', False)
(u'Q8R4X9', u'Q8VDL9', False)
(u'Q922E8', u'O70273', False)
(u'Q8HWA6', u'Q7TNK1', False)
(u'Q8BI45', u'Q3U108', False)
(u'Q920K7', u'P81270', False)
(u'Q3U0G6', u'Q6P3D7', False)
(u'Q62346', u'P41971', False)


Several redirects. Uniprot P31312 redirects to P31257 which is also present. Below fixes the databases, fasta file has to be edited to delete duplicate entry.

In [7]:
from sqlalchemy import exists

for gene_data in session.query(GeneData):
    if gene_data.uniprot != gene_data.fasta.split('|')[1]:
        new_uniprot = gene_data.fasta.split('|')[1]
        gene_data.gene_info.uniprot = new_uniprot

        # if the target of redirection already exists, delete the old entry
        if session.query(exists().where(GeneData.uniprot == gene_data.fasta.split('|')[1])).first()[0]:
            session.query(GeneData).filter(GeneData.uniprot == gene_data.uniprot).delete()
        # else just rename the old entry
        else:
            gene_data.uniprot = new_uniprot

session.commit()

In [10]:
from sqlalchemy import exists

for gene_data in session.query(GeneData):
    if gene_data.uniprot != gene_data.fasta.split('|')[1]:
        print(gene_data.uniprot, gene_data.fasta.split('|')[1],
              session.query(exists().where(GeneData.uniprot == gene_data.fasta.split('|')[1])).first()[0])

### Blast

In [12]:
!makeblastdb -parse_seqids -dbtype prot -in uniprot_seq.fasta -out uniprot_seq_blastdb/uniprot_seq



Building a new DB, current time: 04/10/2015 15:35:51
New DB name:   uniprot_seq_blastdb/uniprot_seq
New DB title:  uniprot_seq.fasta
Sequence type: Protein
Keep Linkouts: T
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 463 sequences in 0.0279899 seconds.


In [13]:
!blastp -db uniprot_seq_blastdb/uniprot_seq -query uniprot_seq.fasta -outfmt 6 -out uniprot_seq.blast

In [12]:
!head uniprot_seq.blast

tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8IKH2|Q8IKH2_PLAF7	100.00	813	0	0	1	813	1	813	0.0	1571
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q5CTD3|Q5CTD3_CRYPI	53.85	52	22	1	64	115	341	390	7e-13	67.0
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8IHT5|Q8IHT5_PLAF7	28.36	134	61	4	15	115	1283	1414	5e-04	38.1
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8IJW6|Q8IJW6_PLAF7	32.79	61	36	1	68	123	776	836	0.003	35.4
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8IJW6|Q8IJW6_PLAF7	27.27	77	52	2	67	143	526	598	0.033	32.3
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8I3U0|Q8I3U0_PLAF7	27.40	73	47	3	47	115	1783	1853	0.039	32.0
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8IHX2|Q8IHX2_PLAF7	25.37	67	48	2	53	117	1345	1411	0.12	30.4
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8IHX2|Q8IHX2_PLAF7	26.67	105	64	4	68	169	290	384	1.1	27.3
tr|Q8IKH2|Q8IKH2_PLAF7	tr|D5GGF2|D5GGF2_TUBMM	47.62	21	11	0	411	431	785	805	3.3	25.8
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8I5I9|Q8I5I9_PLAF7	26.67	60	41	2	76	135	2172	2228	4.8	25.0


### Emboss

In [3]:
def parse_emboss_algn(algn_file):
    """
    Parses identity, similarity and gap percentage.
    """
    with open(algn_file, 'r') as f:
        for line in f:
            if line[:13] == '# Identity:  ':
                identity = float(line.strip().split('(')[1][:-2])
            elif line[:13] == '# Similarity:':
                similarity = float(line.strip().split('(')[1][:-2])
            elif line[:13] == '# Gaps:      ':
                gaps = float(line.strip().split('(')[1][:-2])
    return (identity, similarity, gaps)

In [ ]:
with open('uniprot_seq.algn', 'w') as aligned:

    for seq1 in fasta_iter('uniprot_seq.fasta'):
        with open('_a.fasta', 'w') as f:
            f.write('> ' + seq1[0] + '\n' + seq1[1])

        for seq2 in fasta_iter('uniprot_seq.fasta'):
            with open('_b.fasta', 'w') as f:
                f.write('> ' + seq2[0] + '\n' + seq2[1])

            !water -aformat markx3 -awidth 80 _a.fasta _b.fasta -gapopen 10.0 -gapextend 0.5 -outfile _ab.water 2> /dev/null
            !needle -aformat markx3 -awidth 80 _a.fasta _b.fasta -gapopen 10.0 -gapextend 0.5 -outfile _ab.needle 2> /dev/null

            (local_identity, local_similarity, local_gaps) = parse_emboss_algn('_ab.water')
            (global_identity, global_similarity, global_gaps)= parse_emboss_algn('_ab.needle')
            
            aligned.write('%s\t%s\t%g\t%g\t%g\t%g\t%g\t%g\n' % (seq1[0].split(' ')[0], seq2[0].split(' ')[0],
                                                              local_identity, local_similarity, local_gaps,
                                                              global_identity, global_similarity, global_gaps))

!rm _a.fasta _b.fasta _ab.water _ab.needle

In [2]:
!head uniprot_seq.algn

tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q8IKH2|Q8IKH2_PLAF7	100	100	0	100	100	0
tr|Q8IKH2|Q8IKH2_PLAF7	tr|Q5CTD3|Q5CTD3_CRYPI	20.3	34.7	32.7	16	27.4	47.1
tr|Q8IKH2|Q8IKH2_PLAF7	sp|P41936|HM22_CAEEL	20.2	42.1	21.3	7.6	14.9	68.3
tr|Q8IKH2|Q8IKH2_PLAF7	sp|P14859|PO2F1_HUMAN	20.7	32.9	28	7.8	13.9	67.8
tr|Q8IKH2|Q8IKH2_PLAF7	sp|O70137|ALX3_MOUSE	20	31.4	22.9	0.7	1.7	95.8
tr|Q8IKH2|Q8IKH2_PLAF7	sp|O35137|ALX4_MOUSE	17.5	28.3	36.7	4.4	7.2	83.9
tr|Q8IKH2|Q8IKH2_PLAF7	sp|O35085|ARX_MOUSE	27.9	41.9	25.6	2.1	3.1	94.2
tr|Q8IKH2|Q8IKH2_PLAF7	sp|P97503|NKX32_MOUSE	36.6	56.1	19.5	3.7	6.3	87.9
tr|Q8IKH2|Q8IKH2_PLAF7	sp|P63157|BARH1_MOUSE	24.5	36.5	25.5	5.7	8.9	81.2
tr|Q8IKH2|Q8IKH2_PLAF7	sp|Q8VIB5|BARH2_MOUSE	19.7	35.6	17.4	5.5	8.7	81.1


# domains from PFAM

Use profile's GA gathering cutoffs to set all thresholding

In [4]:
for gene_data in session.query(GeneData):
    domtbl_file = 'pfam_domtbl/' + gene_data.uniprot + '.domtbl'
    fasta = '\"' + gene_data.fasta + '\"'

    !echo $fasta | hmmscan --cut_ga -o /dev/null --domtblout $domtbl_file /global/databases/pfam/v27.0/Pfam-A.hmm -

In [5]:
def get_domains(domtbl_file, accession=False):
    """
    From the domain table file output by HMMscan, obtain the set of all domains found.
    """
    domains = set()
    with open(domtbl_file, 'r') as f:
        for line in f:
            if line[0] != '#':
                if accession:
                    words = line.split()
                    domains.add((words[0], words[1].split('.')[0]))
                else:
                    domains.add(line.split()[0])
    return domains

In [6]:
all_domains = set()
domtbl_files = !ls pfam_domtbl/
for f in domtbl_files:
    all_domains |= get_domains('pfam_domtbl/' + f)
print(len(all_domains))
print(sorted(all_domains))

127
['ACDC', 'AFT', 'AP2', 'ARID', 'AXH', 'Aft1_OSA', 'Ank', 'Ank_2', 'Ank_3', 'Ank_4', 'Ank_5', 'BAF1_ABF1', 'BTB', 'BTD', 'Basic', 'CASP_C', 'CTNNB1_binding', 'Caudal_act', 'Cmyb_C', 'DLL_N', 'DUF1752', 'DUF2028', 'DUF3432', 'DUF3446', 'DUF3454', 'DUF3528', 'DUF4074', 'E2F_TDP', 'EGF', 'EGF_CA', 'ETS_PEA3_N', 'Elf-1_N', 'Engrail_1_C_sig', 'Ets', 'FHA', 'Fork_head', 'Fork_head_N', 'Fungal_trans', 'Fungal_trans_2', 'GABP-alpha', 'GATA', 'GATA-N', 'GCM', 'GCR1_C', 'Gal4_dimer', 'HLH', 'HMG_box', 'HMG_box_2', 'HNF-1A_C', 'HNF-1B_C', 'HNF-1_N', 'HNF_C', 'HSF_DNA-bind', 'Hairy_orange', 'Homeobox', 'Homeobox_KN', 'Homez', 'Hormone_recep', 'Hox9_act', 'HoxA13_N', 'IRF', 'IRF-3', 'JmjC', 'JmjN', 'Jun', 'KRAB', 'KilA-N', 'LAG1-DNAbind', 'LIM', 'LMSTEN', 'MH1', 'MH2', 'Maf_N', 'MamL-1', 'Mannitol_dh', 'Mannitol_dh_C', 'Myb_DNA-bind_6', 'Myb_DNA-binding', 'NDT80_PhoG', 'NOD', 'NODP', 'Neuro_bHLH', 'Notch', 'Nuc_recep-AF1', 'OAR', 'PAP1', 'PAS', 'PAX', 'PBC', 'Pax7', 'Pou', 'RFX1_trans_act', 'RFX

Pfam-A.clans.tsv

This file contains a list of all Pfam-A families that are in clans.
The columns are: Pfam-A accession, clan accession, clan ID, Pfam-A
ID, Pfam-A description.

In [104]:
!wget ftp://ftp.sanger.ac.uk/pub/databases/Pfam/current_release//Pfam-A.clans.tsv.gz
!gunzip Pfam-A.clans.tsv.gz
!head Pfam-A.clans.tsv

--2015-04-07 13:18:29--  ftp://ftp.sanger.ac.uk/pub/databases/Pfam/current_release//Pfam-A.clans.tsv.gz
           => ‘Pfam-A.clans.tsv.gz’
Resolving ftp.sanger.ac.uk (ftp.sanger.ac.uk)... 193.62.203.17
Connecting to ftp.sanger.ac.uk (ftp.sanger.ac.uk)|193.62.203.17|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/databases/Pfam/current_release/ ... done.
==> SIZE Pfam-A.clans.tsv.gz ... 262831
==> PASV ... done.    ==> RETR Pfam-A.clans.tsv.gz ... done.
Length: 262831 (257K) (unauthoritative)

100%[======================================>] 262.831     1009KB/s   in 0,3s   

2015-04-07 13:18:30 (1009 KB/s) - ‘Pfam-A.clans.tsv.gz’ saved [262831]

PF00389	CL0325	Form_Glyc_dh	2-Hacid_dh	D-isomer specific 2-hydroxyacid dehydrogenase, catalytic domain
PF00198	CL0149	CoA-acyltrans	2-oxoacid_dh	2-oxoacid dehydrogenases acyltransferase (catalytic domain)
PF04029	\N	\N	2-ph_phosp	2-phosphosulpholactate phospha

In [7]:
family_clan = dict()
clan_family = dict()

with open('Pfam-A.clans.tsv', 'r') as f:
    for line in f:

        words = line.split('\t')
        if words[2] == '\\N':
            words[2] = 'NA'

        family_clan[words[3]] = words[2]
        if words[2] not in clan_family:
            clan_family[words[2]] = set()
        clan_family[words[2]].add(words[3])

In [8]:
domtbl_files = !ls pfam_domtbl/

with open('pfam.dom', 'w') as pfam_dom:
    with open('pfam.clan', 'w') as pfam_clan:
        for f in domtbl_files:
            uniprot = f.split('.')[0]
            domains = get_domains('pfam_domtbl/' + f)
            pfam_dom.write('>' + uniprot + '\n' + ' '.join(domains) + '\n')
            pfam_clan.write('>' + uniprot + '\n' + ' '.join({(family_clan[dom] if family_clan[dom] != 'NA' else dom)
                                                             for dom in domains}) + '\n')

In [9]:
all_clans = set()
for rec in fasta_iter('pfam.clan'):
    all_clans |= set(rec[1].split())
print(len(all_clans))
print(sorted(all_clans))

97
['6PGD_C', 'ACDC', 'AFT1_OSA', 'ANK', 'ARID', 'AXH', 'BAF1_ABF1', 'BASIC', 'BTD', 'BZIP', 'C2H2-ZF', 'CASP_C', 'CAUDAL_ACT', 'CHEY', 'CMYB_C', 'CTNNB1_BINDING', 'CUPIN', 'DLL_N', 'DUF1752', 'DUF2028', 'DUF3432', 'DUF3446', 'DUF3454', 'DUF3528', 'DUF4074', 'E-SET', 'EGF', 'ELF-1_N', 'ENGRAIL_1_C_SIG', 'ETS_PEA3_N', 'FORK_HEAD', 'FORK_HEAD_N', 'FUNGAL_TRANS', 'GABP-ALPHA', 'GAL4_DIMER', 'GATA-N', 'GCR1_C', 'HAIRY_ORANGE', 'HIS-ME_FINGER', 'HLH', 'HMG-BOX', 'HNF-1A_C', 'HNF-1B_C', 'HNF-1_N', 'HNF_C', 'HORMONE_RECEP', 'HOX9_ACT', 'HOXA13_N', 'HTH', 'IRF', 'JMJN', 'JUN', 'KILA-N', 'KRAB', 'LAG1-DNABIND', 'LIM', 'LMSTEN', 'MAF_N', 'MAML-1', 'MBD-LIKE', 'NADP_ROSSMANN', 'NEURO_BHLH', 'NOD', 'NODP', 'NOTCH', 'NUC_RECEP-AF1', 'OAR', 'P53-LIKE', 'PAP1', 'PAS_FOLD', 'PAX7', 'PBC', 'PKID', 'POZ', 'RAP1_C', 'RFX1_TRANS_ACT', 'SAM', 'SAND', 'SCAN', 'SMAD-FHA', 'SOXP', 'SOX_C_TAD', 'SOX_N', 'SP100', 'SRF-TF', 'STB3', 'STE', 'TBP-LIKE', 'TEA', 'TF_AP-2', 'TF_OTX', 'TRF', 'VHR1', 'WRKY-GCM1', 'ZF-C4

In [10]:
n_clans = len(all_clans)
clan_idx = {clan: i for (i, clan) in enumerate(sorted(all_clans))}

import numpy as np
from scipy.spatial.distance import cosine, euclidean, hamming

with open('pfam.clan.vs', 'w') as clan_vs:

    for seq1 in fasta_iter('pfam.clan'):
        u = np.zeros(n_clans)
        for c in seq1[1].split():
            u[clan_idx[c]] = 1

        for seq2 in fasta_iter('pfam.clan'):
            v = np.zeros(n_clans)
            for c in seq2[1].split():
                v[clan_idx[c]] = 1

            clan_vs.write('%s\t%s\t%g\t%g\t%g\n' % (seq1[0], seq2[0],
                                                    cosine(u, v), euclidean(u, v), hamming(u, v)))

/usr/lib/python2.7/dist-packages/scipy/spatial/distance.py:287: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


In [13]:
!head pfam.clan.vs

A1A546	A1A546	0	0	0
A1A546	A5AA25	nan	1	0.0103093
A1A546	A7RI36	1	1.41421	0.0206186
A1A546	B3RL96	1	1.41421	0.0206186
A1A546	C0H5G5	1	1.41421	0.0206186
A1A546	C0LRF2	0	0	0
A1A546	C6KSY0	1	1.41421	0.0206186
A1A546	D5GGF2	1	1.73205	0.0309278
A1A546	F9WW92	1	1.73205	0.0309278
A1A546	O08580	1	1.73205	0.0309278
